In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score

np.random.seed(42)

In [2]:
iris = load_iris()
x, y = iris.data, iris.target

## Search Space

In [3]:
import sys
sys.path.append('/Users/heyi/Github_Document/HPSearchSpace/HPSearchSpace')

from HPSearchSpace.search_space import SearchSpace

ss = SearchSpace(config_file="example/example.yaml")

print(ss.config)


{'estimators': {'sklearn.svm.SVC': {'C': {'range': [1e-10, 1.0], 'sampler': 'loguniform', 'default': 1.0}, 'kernel': {'values': ['linear'], 'sampler': 'choice', 'default': 'linear'}}, 'sklearn.ensemble.RandomForestClassifier': {'n_estimators': {'range': [10, 1000], 'sampler': 'uniformint', 'default': 10}, 'max_depth': {'range': [2, 32], 'sampler': 'uniformint', 'default': 5}}, 'sklearn.ensemble.GradientBoostingClassifier': {'n_estimators': {'range': [10, 1000], 'sampler': 'uniformint', 'default': 10}, 'max_depth': {'range': [2, 32], 'sampler': 'uniformint', 'default': 5}}, 'sklearn.neighbors.KNeighborsClassifier': {'n_neighbors': {'range': [2, 10], 'sampler': 'uniformint', 'default': 5}}}}


## Hyperopt

In [4]:
from hyperopt import hp
from hyperopt.pyll import scope
from hyperopt import fmin, Trials, tpe
space=ss.get_hyperopt_space()
# space = hp.choice(
#     "classifier", [
#         {
#             "model": SVC,
#             "params": {
#                 "C": hp.loguniform("svm_C", 1e-10, 1),
#                 "kernel": "linear"
#             }
#         },
#         {
#             "model": RandomForestClassifier,
#             "params": {
#                 "n_estimators": scope.int(hp.quniform("rf_n_estimators", 10, 1000, 10)),
#                 "max_depth": scope.int(hp.quniform("rf_max_depth", 2, 32, 1))
#             }
#         },
#         {
#             "model": GradientBoostingClassifier,
#             "params": {
#                 "n_estimators": scope.int(hp.quniform("gb_n_estimators", 10, 1000, 10)),
#                 "max_depth": scope.int(hp.quniform("gb_max_depth", 2, 32, 1))
#             }
#         },
#         {
#             "model": KNeighborsClassifier,
#             "params": {
#                 "n_neighbors": scope.int(hp.quniform("knn_n_neighbors", 2, 10, 1))
#             }
#         }
#     ]
# )

def objective_hp(params):
    model = params["model"]
    params = params["params"]
    model = model(**params)
    score = cross_val_score(model, x, y, n_jobs=-1, cv=3)
    return -score.mean()

trials = Trials()
best = fmin(objective_hp, space, algo=tpe.suggest, max_evals=100, trials=trials)

100%|██████████| 100/100 [00:17<00:00,  5.77trial/s, best loss: -0.9933333333333333]


In [5]:
print(trials.best_trial["result"], trials.best_trial["misc"]["vals"])
print(trials.trials[10]["result"], trials.trials[10]["misc"]["vals"])

{'loss': -0.9933333333333333, 'status': 'ok'} {'classifier': [np.int64(0)], 'gradientboostingclassifier_max_depth': [], 'gradientboostingclassifier_n_estimators': [], 'kneighborsclassifier_n_neighbors': [], 'randomforestclassifier_max_depth': [], 'randomforestclassifier_n_estimators': [], 'svc_C': [np.float64(1.2493219663336799)]}
{'loss': -0.96, 'status': 'ok'} {'classifier': [np.int64(1)], 'gradientboostingclassifier_max_depth': [], 'gradientboostingclassifier_n_estimators': [], 'kneighborsclassifier_n_neighbors': [], 'randomforestclassifier_max_depth': [np.float64(11.0)], 'randomforestclassifier_n_estimators': [np.float64(353.0)], 'svc_C': []}


## Optuna

In [6]:
import optuna
# def objective_optuna(trial_):
#     classifier_name = trial_.suggest_categorical("classifier", [
#         "SVC", "RandomForest", "KNeighbors", "GradientBoosting"
#     ])
#     if classifier_name == "SVC":
#         svc_c = trial_.suggest_float("C", 1e-10, 1, log=True)
#         model = SVC(C=svc_c, kernel="linear")
#     elif classifier_name == "RandomForest":
#         rf_max_depth = trial_.suggest_int("rf_max_depth", 2, 32)
#         rf_n_estimators = trial_.suggest_int("rf_n_estimators", 10, 1000)
#         model = RandomForestClassifier(
#             n_estimators=rf_n_estimators, max_depth=rf_max_depth
#         )
#     elif classifier_name == "KNeighbors":
#         knn_n_neighbors = trial_.suggest_int("knn_n_neighbors", 2, 10)
#         model = KNeighborsClassifier(n_neighbors=knn_n_neighbors)
#     elif classifier_name == "GradientBoosting":
#         gb_max_depth = trial_.suggest_int("gb_max_depth", 2, 32)
#         gb_n_estimators = trial_.suggest_int("gb_n_estimators", 10, 1000)
#         model = GradientBoostingClassifier(
#             n_estimators=gb_n_estimators, max_depth=gb_max_depth
#         )
#     else:
#         raise ValueError("Invalid classifier name")

#     score = cross_val_score(model, x, y, n_jobs=-1, cv=3)
#     return score.mean()

def objective_optuna(trial_):
    model = ss.get_optuna_space(trial_)
    score = cross_val_score(model, x, y, n_jobs=-1, cv=3)
    return score.mean()

study = optuna.create_study(direction="maximize")
study.optimize(objective_optuna, n_trials=100, show_progress_bar=True)

/Users/heyi/AnacondaA/anaconda3/envs/causaltune-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-02-05 13:15:17,454] A new study created in memory with name: no-name-eb77c67d-f92e-4f71-8161-64342340b685
Best trial: 0. Best value: 0.96:   1%|          | 1/100 [00:00<00:15,  6.22it/s]

[I 2025-02-05 13:15:17,614] Trial 0 finished with value: 0.96 and parameters: {'classifier': 'RandomForest', 'n_estimators': 173, 'max_depth': 6}. Best is trial 0 with value: 0.96.


Best trial: 1. Best value: 0.973333:   3%|▎         | 3/100 [00:00<00:42,  2.27it/s]

[I 2025-02-05 13:15:18,250] Trial 1 finished with value: 0.9733333333333333 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 863, 'max_depth': 11}. Best is trial 1 with value: 0.9733333333333333.
[I 2025-02-05 13:15:18,327] Trial 2 finished with value: 0.9666666666666667 and parameters: {'classifier': 'RandomForest', 'n_estimators': 99, 'max_depth': 21}. Best is trial 1 with value: 0.9733333333333333.
[I 2025-02-05 13:15:18,342] Trial 3 finished with value: 0.32 and parameters: {'classifier': 'SVC', 'C': 5.7889153943719186e-08, 'kernel': 'linear'}. Best is trial 1 with value: 0.9733333333333333.


Best trial: 1. Best value: 0.973333:   6%|▌         | 6/100 [00:01<00:21,  4.31it/s]

...

[I 2025-02-05 13:15:36,508] Trial 78 finished with value: 0.9733333333333333 and parameters: {'classifier': 'KNeighbors', 'n_neighbors': 3}. Best is trial 53 with value: 0.9866666666666667.
[I 2025-02-05 13:15:36,596] Trial 79 finished with value: 0.9866666666666667 and parameters: {'classifier': 'KNeighbors', 'n_neighbors': 4}. Best is trial 53 with value: 0.9866666666666667.
[I 2025-02-05 13:15:36,617] Trial 80 finished with value: 0.98 and parameters: {'classifier': 'KNeighbors', 'n_neighbors': 5}. Best is trial 53 with value: 0.9866666666666667.
[I 2025-02-05 13:15:36,640] Trial 81 finished with value: 0.9866666666666667 and parameters: {'classifier': 'KNeighbors', 'n_neighbors': 4}. Best is trial 53 with value: 0.9866666666666667.
[I 2025-02-05 13:15:36,659] Trial 82 finished with value: 0.9866666666666667 and parameters: {'classifier': 'KNeighbors', 'n_neighbors': 4}. Best is trial 53 with value: 0.9866666666666667.
[I 2025-02-05 13:15:36,678] Trial 83 finished with value: 0.9733

Best trial: 53. Best value: 0.986667:  93%|█████████▎| 93/100 [00:19<00:00, 25.61it/s]

[I 2025-02-05 13:15:36,762] Trial 86 finished with value: 0.9733333333333333 and parameters: {'classifier': 'KNeighbors', 'n_neighbors': 3}. Best is trial 53 with value: 0.9866666666666667.
[I 2025-02-05 13:15:36,783] Trial 87 finished with value: 0.32 and parameters: {'classifier': 'SVC', 'C': 2.2141302325427666e-05, 'kernel': 'linear'}. Best is trial 53 with value: 0.9866666666666667.
[I 2025-02-05 13:15:36,834] Trial 88 finished with value: 0.9666666666666667 and parameters: {'classifier': 'RandomForest', 'n_estimators': 33, 'max_depth': 4}. Best is trial 53 with value: 0.9866666666666667.
[I 2025-02-05 13:15:36,854] Trial 89 finished with value: 0.9866666666666667 and parameters: {'classifier': 'KNeighbors', 'n_neighbors': 4}. Best is trial 53 with value: 0.9866666666666667.
[I 2025-02-05 13:15:36,873] Trial 90 finished with value: 0.9466666666666667 and parameters: {'classifier': 'KNeighbors', 'n_neighbors': 2}. Best is trial 53 with value: 0.9866666666666667.
[I 2025-02-05 13:15:

Best trial: 53. Best value: 0.986667: 100%|██████████| 100/100 [00:19<00:00,  5.10it/s]

[I 2025-02-05 13:15:36,970] Trial 94 finished with value: 0.9666666666666667 and parameters: {'classifier': 'KNeighbors', 'n_neighbors': 8}. Best is trial 53 with value: 0.9866666666666667.
[I 2025-02-05 13:15:36,991] Trial 95 finished with value: 0.98 and parameters: {'classifier': 'KNeighbors', 'n_neighbors': 5}. Best is trial 53 with value: 0.9866666666666667.
[I 2025-02-05 13:15:37,011] Trial 96 finished with value: 0.9866666666666667 and parameters: {'classifier': 'KNeighbors', 'n_neighbors': 4}. Best is trial 53 with value: 0.9866666666666667.
[I 2025-02-05 13:15:37,030] Trial 97 finished with value: 0.9733333333333333 and parameters: {'classifier': 'KNeighbors', 'n_neighbors': 3}. Best is trial 53 with value: 0.9866666666666667.
[I 2025-02-05 13:15:37,051] Trial 98 finished with value: 0.9866666666666667 and parameters: {'classifier': 'KNeighbors', 'n_neighbors': 4}. Best is trial 53 with value: 0.9866666666666667.
[I 2025-02-05 13:15:37,070] Trial 99 finished with value: 0.32 a

In [7]:
print(study.best_trial.value, study.best_trial.params)
print(study.trials[10].value, study.trials[10].params)

0.9866666666666667 {'classifier': 'KNeighbors', 'n_neighbors': 4}
0.9733333333333333 {'classifier': 'GradientBoosting', 'n_estimators': 631, 'max_depth': 32}
